In [22]:
import numpy as np
from preprocessing import getData
from pandas import *

In [23]:
def sigmoid(Z):
    return 1/(1 + np.exp(-Z))
def sigmoidDerivate(A):
    return A*(1-A)

In [24]:
def train(train_X,train_Y,train_size,features,classes,weight_seed):
    #Training
    num_hidden1 = 10
    num_hidden2 = 5
    num_iterations = 2000
    alpha = [0.0075,0.0075,0.0075]
    np.random.seed(weight_seed)
    #Initialization of weights
    W1 = np.random.randn(num_hidden1,features)
    b1 = np.zeros((num_hidden1,1))
    W2 = np.random.randn(num_hidden2,num_hidden1)
    b2 = np.zeros((num_hidden2,1))
    W3 = np.random.randn(classes,num_hidden2)
    b3 = np.zeros((classes,1))
    costs = []
    iters = []
    for iteration in range(num_iterations):
        #Forward propagation
        Z1 = np.dot(W1,train_X) + b1
        A1 = sigmoid(Z1)
        Z2 = np.dot(W2,A1) + b2
        A2 = sigmoid(Z2)
        Z3 = np.dot(W3,A2) + b3
        A3 = Y_tilda = sigmoid(Z3)
        #Calculating cost
        costs.append((np.sum(np.power((Y_tilda-train_Y),2)))/(2*train_size*classes))
        iters.append(iteration+1)
        #Backpropagation
        delta_3 = (Y_tilda - train_Y)*sigmoidDerivate(A3)
        delta_2 = W3.T.dot((delta_3))*sigmoidDerivate(A2)
        delta_1 = W2.T.dot((delta_2))*sigmoidDerivate(A1)
        #Update weights
        W3 = W3 - alpha[0]*np.dot(delta_3,A2.T)
        b3 = b3 - alpha[0]*np.sum(delta_3,axis=1,keepdims=True)
        W2 = W2 - alpha[1]*np.dot(delta_2,A1.T)
        b2 = b2 - alpha[1]*np.sum(delta_2,axis=1,keepdims=True)
        W1 = W1 - alpha[2]*np.dot(delta_1,train_X.T)
        b1 = b1 - alpha[2]*np.sum(delta_1,axis=1,keepdims=True)
    
    return W1,b1,W2,b2,W3,b3

In [42]:
def test(test_X,test_Y,W1,b1,W2,b2,W3,b3):
    #Testing
    Z1 = np.dot(W1,test_X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = sigmoid(Z2)
    Z3 = np.dot(W3,A2) + b3
    Y_tilda = sigmoid(Z3)
    test_size = test_X.shape[1]
    count = 0
    TrueZeros = TrueOnes = FalseZeros = FalseOnes = 0
    for i in range(test_size):
        actualClass = np.argmax(test_Y.T[i,:])
        predictedClass = np.argmax(Y_tilda.T[i,:])
        if actualClass == predictedClass:
            count += 1
            if actualClass == 0:
                TrueZeros += 1
            else:
                TrueOnes += 1
        else:
            if actualClass == 0:
                FalseZeros += 1
            else:
                FalseOnes += 1
    conf_mat = ([[TrueZeros, FalseZeros], [FalseOnes, TrueOnes]])
    accuracy = count*100/test_size
    return [accuracy, conf_mat]

In [43]:
#Holdout method -> 70,30
def holdout(X,Y,train_percent):
    train_size = int(train_percent*X.shape[0])
    test_size = X.shape[0] - train_size
    train_X = X[:train_size,:]
    test_X = X[train_size:,:]
    train_Y = Y[:train_size,:]
    test_Y = Y[train_size:,:]
    instances = train_X.shape[0] # equals to train_size
    classes = test_Y.shape[1]
    features = train_X.shape[1]
    train_X = train_X.T
    test_X = test_X.T
    train_Y = train_Y.T
    test_Y = test_Y.T
    W1,b1,W2,b2,W3,b3 = train(train_X,train_Y,train_size,features,classes,1)
    [accuracy, conf_mat] = test(test_X,test_Y,W1,b1,W2,b2,W3,b3)
    print("70-30 Holdout Method :- ")
    print()
    print("Confusion Matrix :-")
    print(DataFrame(conf_mat))
    print()
    print("Accuracy :-", accuracy)

In [44]:
#5 fold cross validation
from sklearn.model_selection import KFold
def k_fold(k):
    kf = KFold(n_splits=5)
    kf.get_n_splits(X)
    fold = 0
    avg_accuracy = 0
    for train_index, test_index in kf.split(X):
        fold += 1
        train_X, test_X = X[train_index], X[test_index]
        train_Y, test_Y = Y[train_index], Y[test_index]
        instances = train_X.shape[0]
        classes = test_Y.shape[1]
        features = train_X.shape[1]
        train_X = train_X.T
        test_X = test_X.T
        train_Y = train_Y.T
        test_Y = test_Y.T
        W1,b1,W2,b2,W3,b3 = train(train_X,train_Y,instances,features,classes,5)
        [accuracy, conf_mat] = test(test_X,test_Y,W1,b1,W2,b2,W3,b3)
        print("Fold %d Accuracy :- %s"%(fold,accuracy),"%")
        print()
        print("Confusion Matrix :-")
        print(DataFrame(conf_mat))
        print()
        avg_accuracy += accuracy

    print("Average Accuracy is :- %s"%(avg_accuracy/fold),"%")

In [45]:
X, Y = getData('data.mat')

In [46]:
holdout(X,Y,0.7)

70-30 Holdout Method :- 

Confusion Matrix :-
     0    1
0  288   45
1   35  277

Accuracy :- 87.59689922480621


In [47]:
k_fold(5)

Fold 1 Accuracy :- 89.76744186046511 %

Confusion Matrix :-
     0    1
0  185   23
1   21  201

Fold 2 Accuracy :- 90.93023255813954 %

Confusion Matrix :-
     0    1
0  193   12
1   27  198

Fold 3 Accuracy :- 92.55813953488372 %

Confusion Matrix :-
     0    1
0  204   17
1   15  194

Fold 4 Accuracy :- 87.41258741258741 %

Confusion Matrix :-
     0    1
0  186   22
1   32  189

Fold 5 Accuracy :- 88.81118881118881 %

Confusion Matrix :-
     0    1
0  207   26
1   22  174

Average Accuracy is :- 89.89591803545292 %
